In [3]:
from pyniryo import *
import pygame
import numpy as np
import time
import math

pygame 2.1.0 (SDL 2.0.16, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
robot_ip_address = "10.10.10.10"
screen_size = (640, 480)
screen_title = 'Niryo Robot Control'
# robot_ip_address = "169.254.200.200"

In [17]:
screen = pygame.display.set_mode(screen_size)
pygame.display.set_caption(screen_title)

In [18]:
robot = NiryoRobot(robot_ip_address)


Connected to server (10.10.10.10) on port: 40001



In [19]:
robot.calibrate_auto()

In [20]:
move_speed = 0.1
rot_speed = 0.2

In [21]:
robot.move_to_home_pose()

In [8]:
pose = robot.get_pose()
pose

x = 0.1339, y = 0.0001, z = 0.1647
roll = -0.001, pitch = 1.001, yaw = 0.001

In [ ]:
done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
    pose = robot.get_pose()
    # Get the current state of the WASD keys
    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        pose.x +=0.075
        robot.move_pose(pose)
    if keys[pygame.K_s]:
        pose.x -=0.075
        robot.move_pose(pose)
    if keys[pygame.K_a]:
        pose.y +=0.075
        robot.move_pose(pose)    
    if keys[pygame.K_d]:
        pose.y -=0.075
        robot.move_pose(pose)
    if keys[pygame.K_SPACE]:
        pose.z +=0.075
        robot.move_pose(pose)
    if keys[pygame.K_LCTRL]:
        pose.z -=0.075
        robot.move_pose(pose)
    if keys[pygame.K_q]:
        pose.roll -=0.5
        robot.move_pose(pose)
    if keys[pygame.K_e]:
        pose.roll +=0.5
        robot.move_pose(pose)
    if keys[pygame.K_o]:
        robot.open_gripper(speed=500)
    if keys[pygame.K_p]:
        robot.close_gripper(speed=500)
    # Update the screen
    screen.fill((255, 255, 255))
    pygame.display.update()

    # Wait for a short period of time to avoid overloading the robot with commands
    time.sleep(0.01)

In [14]:
robot.close_connection()


Disconnected from robot



In [16]:
base_model = PoseObject(
x=0.5, y=0, z=0.25,
roll=-0, pitch=0, yaw=0
)
robot.move_pose(base_model)

In [ ]:
rad = robot.get_joints()
deg = []
deg = math.degrees(rad)

[-0.0007288597570993538,
 -1.2958018047396962,
 1.2672217058481852,
 -0.0014413271980928677,
 0.029052981380034204,
 0.0016266343776782932]

In [4]:
rad = [-0.0007288597570993538,
 -1.2958018047396962,
 1.2672217058481852,
 -0.0014413271980928677,
 0.029052981380034204,
 0.0016266343776782932]
deg = []
for i in rad:
    deg.append(math.degrees(i))
deg

[-0.041760587938723316,
 -74.24397449701979,
 72.60645545246967,
 -0.08258196534813768,
 1.664613215348126,
 0.09319928465185537]

In [13]:
robot.move_joints(0, 0, 0, 0, 0, 0)

In [ ]:
robot.get_pose()

In [ ]:
robot.update_tool()

In [8]:
robot.close_gripper(speed=500)

In [20]:
robot.open_gripper(speed=500)

In [8]:
pick_pose = PoseObject(
x=0.1, y=0.2, z=0.4,
roll=0, pitch=0, yaw=0
)

robot.move_pose(pick_pose)
# robot.move_pose(pick_pose)


In [21]:
joints_read = robot.get_joints()
joints_read

[-0.0007288597570993538,
 0.49940895727663126,
 -1.2506181983468665,
 9.265358979293481e-05,
 -0.0031606151655640957,
 0.0016266343776782932]